<p>
<font size='5' face='Georgia, Arial'>IIC-2233 Apunte Programación Avanzada</font><br>
<font size='1'> Equipo Docente IIC2233 2019 al 2023</font>
</p>

# Tabla de contenidos

1. [Ejecución de código de ejemplo del material que usa PyQt](#Ejecución-de-código-de-ejemplo-del-material-que-usa-PyQt)
2. [*Threads* y PyQt](#Threads-y-PyQt)
3. [*QThread*](#QThread)
    1. [`QMutex`](#QMutex)
4. [Ejemplo Aplicado](#Ejemplo-Aplicado)

## Ejecución de código de ejemplo del material que usa PyQt

Lamentablemente, Jupyter no fue creado con la intención de ejecutar código de GUIs de escritorio, por lo que se hace difícil ejecutar e interactuar con las interfaces a través de esta herramienta. Para entender los ejemplos de código relacionados con PyQt, **recomendamos fuertemente** que ejecutes los *scripts* de interfaces gráficas desde tu propio computador, y **NO** en este *notebook*. Para esto, se agregó un directorio llamada `scripts` que tiene cada código que se mostrará en este y los siguientes _notebooks_. 

Además, dentro del directorio `scripts` hay otro llamado `PyQt5` donde están exactamente los mismos ejemplos, pero utilizando `PyQt5` en vez de `PyQt6`.

## *Threads* y PyQt

Esta semana pasaremos de una programación secuencial con un único hilo (*thread*) de ejecución con solo interacción por consola y archivos, a un modelo *multithreaded* orientado a interactuar con el usuario mediante distintos tipos de *inputs* gráficos.

El uso de *theading* en conjunto a PyQt es absolutamente compatible, y, si queremos hacer programas de amyor complejidad, a veces es necesario.

Al igual que para programas con un solo *thread*, podemos hacer uso de señales personalizadas en una aplicación *multithreaded*. El siguiente ejemplo muestra cómo crear una señal que controla la acción del *thread* sobre el formulario mediante el método `actualizar_labels()`. El *thread* por su parte, recibe como parámetro la señal y emite mensajes.

Este código se encuentra en el archivo `1-pyqt-qthreads-ejemplo_1.py`

```python

import sys
from threading import Thread
from time import sleep
from PyQt6.QtCore import pyqtSignal
from PyQt6.QtWidgets import QApplication, QWidget, QLabel, QPushButton


class MiThread(Thread):
    """
    Esta clase representa un thread personalizado que será utilizado durante
    la ejecución de la GUI.
    """

    def __init__(self, senal_actualizar):
        super().__init__()
        self.senal_actualizar = senal_actualizar

    def run(self):
        for i in range(10):
            sleep(0.5)
            self.senal_actualizar.emit(str(i))
        self.senal_actualizar.emit("Status: thread terminado")


class MiVentana(QWidget):
    # Creamos una señal para manejar la respuesta del thread
    senal_thread = pyqtSignal(str)

    def __init__(self):
        super().__init__()
        self.thread = None
        # Conectamos la señal del thread al método que maneja
        self.senal_thread.connect(self.actualizar_labels)
        self.init_gui()

    def init_gui(self):
        # Configuramos los widgets de la interfaz
        self.label = QLabel("Status: esperando thread", self)
        self.boton = QPushButton("Ejecutar Thread", self)
        self.boton.clicked.connect(self.ejecutar_threads)

        self.label.setGeometry(10, 10, 230, 30)
        self.boton.setGeometry(10, 50, 230, 30)

        # Configuramos las propiedades de la ventana.
        self.setWindowTitle("Ejemplo thread")
        self.setGeometry(50, 50, 250, 200)
        self.show()

    def ejecutar_threads(self):
        """
        Este método crea un thread cada vez que se presiona el botón en la
        interfaz. El thread recibirá como argumento la señal sobre la cual
        debe operar.
        """
        if self.thread is None or not self.thread.is_alive():
            self.thread = MiThread(self.senal_thread)
            self.thread.start()

    def actualizar_labels(self, evento):
        """
        Este método actualiza el label según los datos enviados desde el
        thread através del objeto evento. Para este ejemplo, el método
        recibe el evento, pero podría también no recibir nada.
        """
        self.label.setText(evento)

```

Con esto, y lo que sabemos del módulo `threading`, nos basta para generar comportamiento *multithread* en conjunto con nuestras interfaces, pero hay más. Debido a que PyQt se construye sobre la **arquitectura basada en manejo de eventos**, *threads* comunes y corrientes pueden generar problemas de concurrencia con esta arquitectura, por lo que PyQt provee su propia implementación de *threads*: el `QThread`.

## QThread

En términos simples, es un `Thread` como los del módulo `threading`. Provee prácticamente las mismas funcionalidades que ya conocemos, pero está creado dentro del ambiente de PyQt, por lo que es mucho más compatible con su arquitectura interna. Es parte del módulo `PyQt6.QtCore`, por lo que deberás importarlo desde ese módulo. A continuación se muestra una recreación del ejemplo anterior utilizando `QThread` en vez de `Thread`.

Este código se encuentra en el archivo `1-pyqt-qthreads-ejemplo_2.py`

```python
import sys
from time import sleep
from PyQt6.QtCore import pyqtSignal, QThread
from PyQt6.QtWidgets import QApplication, QWidget, QLabel, QPushButton


class MiThread(QThread):
    """
    Esta clase representa un thread personalizado que será utilizado durante
    la ejecución de la GUI.
    """

    def __init__(self, senal_actualizar):
        super().__init__()
        self.senal_actualizar = senal_actualizar

    def run(self):
        for i in range(10):
            sleep(0.5)
            self.senal_actualizar.emit(str(i))

        sleep(0.5)
        self.senal_actualizar.emit("Status: Qthread terminado")


class MiVentana(QWidget):
    # Creamos una señal para manejar la respuesta del thread
    senal_thread = pyqtSignal(str)

    def __init__(self):
        super().__init__()
        self.thread = None
        # Conectamos la señal del thread al método que maneja
        self.senal_thread.connect(self.actualizar_labels)

        self.init_gui()

    def init_gui(self):
        # Configuramos los widgets de la interfaz
        self.label = QLabel("Status: esperando Qthread", self)
        self.boton = QPushButton("Ejecutar QThread", self)
        self.boton.clicked.connect(self.ejecutar_threads)

        self.label.setGeometry(10, 10, 230, 30)
        self.boton.setGeometry(10, 50, 230, 30)

        # Configuramos las propiedades de la ventana.
        self.setWindowTitle("Ejemplo Qthreads")
        self.setGeometry(50, 50, 250, 200)
        self.show()

    def ejecutar_threads(self):
        """
        Este método crea un thread cada vez que se presiona el botón en la
        interfaz. El thread recibirá como argumento la señal sobre la cual
        debe operar.
        """
        # Aquí debemos ocupar isRunning en lugar de is_alive
        if self.thread is None or not self.thread.isRunning():
            self.thread = MiThread(self.senal_thread)
            self.thread.start()

    def actualizar_labels(self, evento):
        """
        Este método actualiza el label según los datos enviados desde el
        thread através del objeto evento. Para este ejemplo, el método
        recibe el evento, pero podría también no recibir nada.
        """
        self.label.setText(evento)

```

Se puede apreciar que se define de la misma forma que un `Thread`: se llama a `__init__` de la superclase de la cual se hereda, se define el método `run` que define el comportamiento al ser ejecutado, y este se ejecuta al llamar el método `start`. Una diferencia leve que se puede notar es que el método `is_alive` no existe en `QThread`, pero existe un equivalente llamado `isRunning`. En general, se puede encontrar un equivalente para todo método de `Thread` en `QThread`, y se pueden encontrar buscando documentación de la clase.

Ahora veremos un ejemplo con múltiples `QThread` ejecutándose. Este código se encuentra en el archivo `1-pyqt-qthreads-ejemplo_3.py`

```python

import sys
from time import sleep
from PyQt6.QtCore import pyqtSignal, QThread
from PyQt6.QtWidgets import QApplication, QWidget, QLabel, QPushButton


class MiThread(QThread):
    # Se define para la clase MiThread,
    # para que cada instancia tenga una propia
    senal_actualizar = pyqtSignal(int, str)

    def __init__(self, i, tiempo):
        super().__init__()
        self.indice = i
        self.tiempo = tiempo

    def run(self):
        for i in range(10):
            sleep(self.tiempo)
            self.senal_actualizar.emit(self.indice, str(i))

        sleep(self.tiempo)
        self.senal_actualizar.emit(self.indice, "Status: Qthread terminado")


class MiVentana(QWidget):
    def __init__(self):
        super().__init__()
        self.threads = []
        self.init_gui()

    def init_gui(self):
        # Configuramos los widgets de la interfaz
        # Definimos un montón de labels que corresponderán a un thread cada uno
        self.labels = {i: QLabel("Status: esperando Qthread", self) for i in range(1, 6)}
        self.boton = QPushButton("Ejecutar QThreads", self)
        self.boton.clicked.connect(self.ejecutar_threads)

        for i in range(1, 6):
            self.labels[i].setGeometry(10, (i - 1) * 30, 230, 30)

        self.boton.setGeometry(10, 150, 230, 30)
        # Configuramos las propiedades de la ventana.
        self.setWindowTitle("Ejemplo Qthreads")
        self.setGeometry(50, 50, 250, 200)
        self.show()

    def ejecutar_threads(self):
        """
        Este método crea cinco threads cada vez que se presiona el botón en la
        interfaz. Los threads recibirán como argumento el índice del label
        que les corresponde y el tiempo que toman entre cada iteración.
        """
        if any([thread.isRunning() for thread in self.threads]):
            return

        self.threads = []
        for i in range(1, 6):
            thread = MiThread(i, i / 10)
            # Se conecta la señal emitida por el thread a un método
            # de la ventana
            thread.senal_actualizar.connect(self.actualizar_labels)
            self.threads.append(thread)
            thread.start()

    def actualizar_labels(self, indice, texto):
        """
        Este método actualiza el label correspondiente según los datos
        enviados desde un thread através del índice y aplica el texto.
        """
        self.labels[indice].setText(texto)

```

### `QMutex`

Tal como vimos anteriormente, PyQt ofrece `QThread` como una versión propia del `Thread`. ESto mismo ocurre con los _locks_. Este objeto que nos permite controlar al acceso de múltiples _threads_ a una zona crítica de código tambien tiene su versión en PyQt. Este objeto se llama `QMutex` y mediante los métodos `lock()` y `unlock()` podemos garantizar que cierto fragmente de código solo sea ejecutado por 1 _thread_ a la vez.

A continuación mostraremos un ejemplo donde usaremos 6 _threads_ que actualizarán la ventana en diferentes tiempos. No obstante, el primer `for` que actualiza la ventana estará dentro de un `QMutex`, por lo que solo 1 _thread_ podrá ejecutar dicho código a la vez, mientras que un segundo `for` que actualiza la ventana, no tendrá. `QMutex`. Esto hará que mientras uno o más threads estén ejecutando su segunda actualización, solo 1 podrá realizar la primera.

Este código se encuentra en el archivo `3-pyqt-miscelaneo-ejemplo_5.py`.

```python

import sys
from time import sleep
from PyQt6.QtCore import pyqtSignal, QThread, QMutex
from PyQt6.QtWidgets import QApplication, QWidget, QLabel, QPushButton


class MiThread(QThread):
    senal_actualizar = pyqtSignal(int, str)

    def __init__(self, i, tiempo, mutex):
        super().__init__()
        self.indice = i
        self.tiempo = tiempo
        self.mutex = mutex

    def run(self):
        self.mutex.lock()
        for i in range(10):
            sleep(self.tiempo)
            self.senal_actualizar.emit(self.indice, str(i) + " - crítico")
        self.mutex.unlock()

        for i in range(100):
            sleep(self.tiempo)
            self.senal_actualizar.emit(self.indice, str(i) + " - no crítico")

        sleep(self.tiempo)
        self.senal_actualizar.emit(self.indice, "Status: Qthread terminado")


class MiVentana(QWidget):
    def __init__(self):
        super().__init__()
        self.threads = []
        self.mutex = QMutex()
        self.init_gui()

    def init_gui(self):
        # Configuramos los widgets de la interfaz
        # Definimos un montón de labels que corresponderán a un thread cada uno
        self.labels = {
            i: QLabel("Status: esperando Qthread", self) for i in range(1, 6)
        }
        self.boton = QPushButton("Ejecutar QThreads", self)
        self.boton.clicked.connect(self.ejecutar_threads)

        for i in range(1, 6):
            self.labels[i].setGeometry(10, (i - 1) * 30, 230, 30)

        self.boton.setGeometry(10, 150, 230, 30)
        # Configuramos las propiedades de la ventana.
        self.setWindowTitle("Ejemplo QMutex")
        self.setGeometry(50, 50, 250, 200)
        self.show()

    def ejecutar_threads(self):
        """
        Este método crea cinco threads cada vez que se presiona el botón en la
        interfaz. Los threads recibirán como argumento el índice del label
        que les corresponde y el tiempo que toman entre cada iteración.
        """
        if any([thread.isRunning() for thread in self.threads]):
            return

        self.threads = []
        for i in range(1, 6):
            thread = MiThread(i, i / 10, self.mutex)
            # Se conecta la señal emitida por el thread a un método
            # de la ventana
            thread.senal_actualizar.connect(self.actualizar_labels)
            self.threads.append(thread)
            thread.start()

    def actualizar_labels(self, indice, texto):
        """
        Este método actualiza el label correspondiente según los datos
        enviados desde un thread através del índice y aplica el texto.
        """
        self.labels[indice].setText(texto)

```

## Ejemplo Aplicado

Ahora, vamos a aplicar conceptos de modularización, _back-end_ y _front-end_, `qthreads` para generar el siguiente ejemplo:

![](img/ejemplo_aplicado.gif)


En este ejemplo, tendremos diferentes cuadrados que van de un extremo a otro a diferente velocidad. Además, se puede hacer click en un cuadrado para cambiar su color y detener su movimiento. (Mini desafío: intenta detener todos los cuadrados).

Para esto, dentro del directorio `scripts` hay un directorio llamado `ejemplo_aplicado`. En este tenemos los siguientes archivos:

* `main.py`: es el archivo principal a ejecutar.
* `parametros_general.py`: son constantes que se ocuparán tanto en _back-end_ y _front-end_.

**backend**
* `backend/logica_qthread.py`: es el archivo principal del _back-end_ donde se aplica toda la lógica del movimiento de los cuadrados. En este archivo se ocupan `QThread` para modelar cada cuadrado.
* `backend/logica_qtimer.py`: es el mismo archivo que `logica_qthread.py` pero se utilizan `QTimer` en vez de `QThread`. Este archivo se recomienda ignorar para este ejemplo. Se utilizará en el contenido número 2.
* `backend/parametros_backend.py`: son constantes que se ocuparán únicamente en _back-end_.

**frontend**
* `frontend/ventana.py`: es el archivo principal del _front-end_ donde se crea la ventana y todo elemento visual.
* `frontend/parametros_backend.py`: son constantes que se ocuparán únicamente en _front-end_.

Para la correcta ejecución de este archivo, debes abrir tu terminal (usar WSL en caso de Windows) y desplazarte con `cd` hasta el directorio `scripts/ejemplo_aplicado/` y luego escribir `python3 main.py`.